# Animation of the PSF as a function of bandwidth

_M Kenworthy_ Leiden Observatory kenworthy@strw.leidenuniv.nl

Based on hcipy tutorials https://docs.hcipy.org/0.3.1/tutorials/index.html and examples, and http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-as-interactive-javascript-widgets/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from hcipy import *
from scenes import *

%matplotlib inline

anim_version='v2'

In [ ]:
pupil_grid = make_pupil_grid(256)

telescope_pupil_generator = make_magellan_aperture(normalized=True)

telescope_pupil = telescope_pupil_generator(pupil_grid)

im = imshow_field(telescope_pupil, cmap='gray')
plt.colorbar()
plt.xlabel('x / D')
plt.ylabel('y / D')
plt.show()

In [ ]:
wavefront = Wavefront(telescope_pupil)

focal_grid = make_focal_grid(q=8, num_airy=16)
prop = FraunhoferPropagator(pupil_grid, focal_grid)

focal_image = prop.forward(wavefront)

imshow_field(np.log10(focal_image.intensity / focal_image.intensity.max()), vmin=-5)
plt.xlabel('Focal plane distance [$\lambda/D$]')
plt.ylabel('Focal plane distance [$\lambda/D$]')
plt.colorbar()
plt.show()

In [ ]:
f, (a0, a1) = plt.subplots(1,2, figsize=(12,6))

imshow_field(telescope_pupil, ax=a0)
imshow_field(np.log10(focal_image.intensity / focal_image.intensity.max()), vmin=-5, ax=a1)


In [ ]:

from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
fig, ax = plt.subplots()
bandwidth = 0.2
focal_total = 0

for wlen in np.linspace(1 - bandwidth / 2., 1 + bandwidth / 2., 11):
    wavefront = Wavefront(telescope_pupil, wlen)
    focal_total += prop(wavefront).intensity

im = imshow_field(np.log10(focal_total / focal_total.max()), vmin=-5, ax=ax)

te = ax.text(0.23,0.9,"$\Delta \lambda/\lambda ={:.2f}$".format(bandwidth),
             fontsize=20, color='white',
             horizontalalignment='left', verticalalignment='center',
             transform=ax.transAxes)

cbar = plt.colorbar()
cbar.set_label('$\log10$ (Normalised PSF intensity)', rotation=90)

ax.text(0.95,0.05,"@mattkenworthy", fontsize=12, color='white',
            horizontalalignment='right', verticalalignment='bottom',
            transform=ax.transAxes)


ax.text(0.05,0.05,anim_version, fontsize=12, color='white',
            horizontalalignment='left', verticalalignment='bottom',
            transform=ax.transAxes)

In [ ]:
total_time_animation = 11 # seconds
frame_rate = 10          # frames per second
total_frames = total_time_animation * frame_rate # seconds
interval = np.int(1000./frame_rate) # milliseconds

bandwidth = Stage()
bandwidth.add(Act(0.01,0.5,5,'sig',10))
bandwidth.add(Act(0.5,0.01,6,'sig',10))
print(np.linspace(0,bandwidth.total_time(),total_frames))
print(bandwidth.total_time())
myb = bandwidth.t(np.linspace(0,bandwidth.total_time()-0.05,total_frames))
print(myb)

In [ ]:
def animate(i):    
    focal_total = 0
    curr_band = myb[i]
    for wlen in np.linspace(1 - curr_band / 2., 1 + curr_band / 2., 11):
        wavefront = Wavefront(telescope_pupil, wlen)
        focal_total += prop(wavefront).intensity
        
    im = imshow_field(np.log10(focal_total / focal_total.max()), vmin=-5, ax=ax)
    te.set_text("$\Delta \lambda/\lambda ={:.2f}$".format(curr_band))
    return [im]

anim = animation.FuncAnimation(fig, animate,
                               frames=total_frames, interval=interval, 
                               blit=True,
                               repeat=True, repeat_delay=1000)

HTML(anim.to_html5_video())

In [ ]:
###HTML(anim.to_jshtml())


In [ ]:
anim.save('./anim_psf_vs_bandwidth_{}.gif'.format(anim_version), writer='imagemagick', fps=frame_rate)
print('done!')